In [ ]:
import xarray as xr
import os
import json
import socket

import matplotlib.pyplot as plt

import sys
sys.path.append('..//')
from utils_energy_analysis import *

import dask
from dask.distributed import Client
dask.config.set({
    'distributed.worker.memory.target': 0.6,  # fraction of memory to start spilling
    'distributed.worker.memory.spill': 0.7,   # fraction to spill to disk
    'distributed.worker.memory.pause': 0.8,   # fraction to pause worker
})

client = Client(processes=True, n_workers=20, threads_per_worker=1)

In [ ]:
model = 'dummy_rectangle'
with open('..//config.json', 'r') as file:
    mitgcm_config = json.load(file)[socket.gethostname()][model]

In [ ]:
folder_path = os.path.dirname(mitgcm_config['datapath'])
output_folder = os.path.join(folder_path, "seiche_analysis")

cutoff1_hr = 14
cutoff2_hr = 8

cutoff1 = 1/(cutoff1_hr * 3600)
cutoff2 = 1/(cutoff2_hr * 3600)

path_energy_budget = os.path.join(folder_path, f'energy_budget')
path_cutoff_folder = os.path.join(output_folder, f'{cutoff2_hr}_{cutoff1_hr}h')

## Get filtered velocities

In [ ]:
useiche = xr.open_dataset(os.path.join(path_cutoff_folder, rf"u_filtered_{cutoff2_hr}-{cutoff1_hr}h.zarr")).UVEL
vseiche = xr.open_dataset(os.path.join(path_cutoff_folder, rf"v_filtered_{cutoff2_hr}-{cutoff1_hr}h.zarr")).VVEL
wseiche = xr.open_dataset(os.path.join(path_cutoff_folder, rf"w_filtered_{cutoff2_hr}-{cutoff1_hr}h.zarr")).WVEL

## Reformat datasets

In [ ]:
aligned_useiche = useiche.rename({'XG':'XC'})
aligned_useiche['XC'] = wseiche['XC']

aligned_vseiche = vseiche.rename({'YG':'YC'})
aligned_vseiche['YC'] = wseiche['YC']

aligned_wseiche = wseiche.rename({'Zl':'Z'})
aligned_wseiche['Z'] = useiche['Z']

## Compute kinetic energy

In [ ]:
ke_mj_seiche = compute_ke(aligned_useiche, aligned_vseiche, aligned_wseiche, 200, 200, aligned_useiche.drF)

In [ ]:
ke_mj_seiche.isel(time=5, Z=5).plot()

## Get kinetic energy over the entire lake

In [ ]:
df_ke = ke_mj_seiche.sum(dim=['XC','YC','Z']).to_dataframe(name='ke_mj_seiche')['ke_mj_seiche']

In [ ]:
df_ke.plot()
plt.title('Kinetic Energy seiche (surface)')
plt.ylabel('KE [MJ])')
plt.savefig(os.path.join(path_cutoff_folder, "seiche_ke"))

In [ ]:
df_ke.reset_index().to_csv(os.path.join(path_energy_budget, rf"ke_seiche_{cutoff2_hr}-{cutoff1_hr}h.csv"))

## Plot and save maps

In [ ]:
path_map = os.path.join(path_cutoff_folder, r"map_ke_surface")
os.makedirs(path_map, exist_ok=True)
for i in range(ke_mj_seiche.sizes['time']):
    plt.close('all')
    plt.figure(figsize=(15,7))
    ke_mj_seiche.isel(time=i).plot(vmax=1)
    plt.title(ke_mj_seiche.time.isel(time=i).values)
    plt.savefig(os.path.join(path_map, rf"ke_surface_time{i}_{cutoff2_hr}-{cutoff1_hr}h.png"))